# Question 1

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 15:31:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
 spark.version

'3.5.0'

# Question 2

In [3]:
import pandas as pd

/Users/khalileldaou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/khalileldaou/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv')

In [5]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [6]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [7]:
from pyspark.sql import types

In [8]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)]
    )

In [9]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [10]:
df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 13, 22), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 43), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 37, 20), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 56, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 57, 47), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=

In [11]:
df = df.repartition(6)

In [13]:
df.write.parquet('fhv_trips/2019/10')

In [14]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



# Question 3

In [15]:
from pyspark.sql import functions as F

In [16]:
df = df \
        .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
        .withColumn('dropoff_date', F.to_date(df.dropOff_datetime)) \
        .select('dispatching_base_num', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')

In [17]:
df.filter(df.pickup_date == '2019-10-15').count()

62610

# Question 4

In [21]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [22]:
df = df.repartition(6)

In [23]:
df.registerTempTable('fhv_trips')

In [24]:
spark.sql("""
SELECT
    DATEDIFF(hour, pickup_datetime, dropOff_datetime)
FROM
    fhv_trips
ORDER BY 1 DESC
LIMIT
    10
""").show()

+------------------------------------------------------+
|timestampdiff(hour, pickup_datetime, dropOff_datetime)|
+------------------------------------------------------+
|                                                631152|
|                                                631152|
|                                                 87672|
|                                                 70128|
|                                                  8794|
|                                                  8784|
|                                                  1464|
|                                                  1056|
|                                                  1056|
|                                                   793|
+------------------------------------------------------+



# Question 6

In [25]:
df_zones = pd.read_csv('taxi+_zone_lookup.csv')

In [26]:
df_zones.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [27]:
spark.createDataFrame(df_zones).schema

StructType([StructField('LocationID', LongType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [28]:
zones_schema = types.StructType([
types.StructField('LocationID', types.IntegerType(), True), 
types.StructField('Borough', types.StringType(), True), 
types.StructField('Zone', types.StringType(), True), 
types.StructField('service_zone', types.StringType(), True)])

In [29]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv('taxi+_zone_lookup.csv')

In [30]:
df_zones.registerTempTable('zones')

/Users/khalileldaou/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [31]:
spark.sql("""
SELECT
    DATEDIFF(hour, pickup_datetime, dropOff_datetime)
FROM
    fhv_trips 
ORDER BY 1 DESC
LIMIT
    10
""").show()

+------------------------------------------------------+
|timestampdiff(hour, pickup_datetime, dropOff_datetime)|
+------------------------------------------------------+
|                                                631152|
|                                                631152|
|                                                 87672|
|                                                 70128|
|                                                  8794|
|                                                  8784|
|                                                  1464|
|                                                  1056|
|                                                  1056|
|                                                   793|
+------------------------------------------------------+



In [32]:
df_result = df.join(df_zones, df.PUlocationID == df_zones.LocationID)

In [33]:
group_by_dataframe = df_result.groupBy('Zone').count()

In [34]:
from pyspark.sql.functions import asc

In [35]:
group_by_dataframe.sort(asc("count")).show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
|    Inwood Hill Park|   39|
|       Willets Point|   47|
|Forest Park/Highl...|   53|
|  Brooklyn Navy Yard|   57|
|        Crotona Park|   62|
|        Country Club|   77|
|     Freshkills Park|   89|
|       Prospect Park|   98|
|     Columbia Street|  105|
|  South Williamsburg|  110|
+--------------------+-----+
only showing top 20 rows

